In [1]:
import pandas as pd

In [25]:
quest_user = "comment payer"

# I. Classification de la question user en domaine (to be CenterPark or not)

In [3]:
# from joblib import load
# classifier_domaine = load('model_classif_domaine.joblib')

In [4]:
# theme_quest_user = classifier_theme.predict(X_quest_user_clean_vectorized_tfidf)

##  A. Si dans le domaine

### 1.Classification de la question user en themes

In [5]:
dic_code_theme = {"Préparer mon séjour": 1,
                  "Réserver et payer": 2,
                  "Gérer ma réservation": 3,
                  "Mon séjour": 4,
                  "Assurances": 5}
dic_decode_theme = {val: key for key, val in dic_code_theme.items()}

In [6]:
import nltk
import string
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('french')
nlp = spacy.load('fr_core_news_sm')

sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
sw.sort()

def lemmatise_text(text):
    lst_lematised = [token.lemma_ for token in nlp(text)] 
    return ' '.join(lst_lematised).lower()


def stem_text(text):
    lst_stemmerised = [stemmer.stem(token) for token in word_tokenize(text)]    
    return ' '.join(lst_stemmerised)


def substitute_punctuation(text):
    return ' '.join(text.replace("'", ' ').translate(str.maketrans('', '', string.punctuation)).split())


def supp(text):
    return text.replace("«", "").replace("’", "").replace("•", "").replace("®", "")

In [7]:
from joblib import load
vectoriser_theme = load('vectorizer_classif_theme.joblib')
classifier_theme = load('model_classif_theme.joblib') 

Using TensorFlow backend.


In [26]:
quest_user_clean = supp(substitute_punctuation(stem_text(lemmatise_text(quest_user))))

In [27]:
X_quest_user = pd.Series(quest_user_clean)

In [28]:
X_quest_user_clean_vectorized_tfidf = vectoriser_theme.transform(X_quest_user)

Redimension car NN

In [29]:
XX_quest_user = X_quest_user_clean_vectorized_tfidf.toarray().reshape(X_quest_user_clean_vectorized_tfidf.shape[0],1,
                                                X_quest_user_clean_vectorized_tfidf.shape[1])

Prédiction avec le classifieur

In [30]:
pred_proba = classifier_theme.predict(XX_quest_user)

Transformation proba en classe

In [31]:
import numpy as np
idx = np.argmax(pred_proba, axis=-1)
YY_pred = np.zeros( pred_proba.shape )
YY_pred[ np.arange(YY_pred.shape[0]), idx] = 1

In [32]:
theme_quest_user = list(YY_pred[0]).index(1) +1
theme_quest_user

2

### 2. Trouver la question de la FAQ la plus proche/similaire de la question user

### 2. Trouver la question de la FAQ la plus proche/similaire de la question user

In [85]:
theme_quest_user = classifier_theme.predict(X_quest_user_clean_vectorized_tfidf)

In [86]:
theme_quest_user

array([2], dtype=int64)

### 2. Trouver la question de la FAQ la plus proche/similaire de la question user

### 3. Renvoyer la plus réponse de cette question FAQ

## B. Pas dans le domaine : générer réponse originale avec le GAN sur text (partie Mathilde)